In [1]:
import sys

sys.path.insert(0, '..')

from processing.audio.SIWhisperModel import SIWhisperModel
from SIWeaviateClient import SIWeaviateClient
from processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/../app/schemas.py:80: GenericBeforeBaseModelWarning: Classes should inherit from `BaseModel` before generic classes (e.g. `typing.Generic[T]`) for pydantic generics to work properly.
  class ChunkWithScore(Generic[ChunkType], BaseDocumentChunk):
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erwan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
urls = [
    "https://www.youtube.com/watch?v=re__12nH2ro",
    # "https://www.youtube.com/watch?v=FSbEorqTtMU",
    # "https://www.youtube.com/watch?v=lds6ExALbjk",
    # "https://www.youtube.com/watch?v=0oB2oBj5Ex4",
    # "https://www.youtube.com/watch?v=p5JR8fsEzR4",
]
with SIWeaviateClient() as client:
    for url in urls:
        YoutubeProcessor(client=client, whisper=whisper, youtube_url=url)

AUDIO CODEC aac


whisper-medium: 100%|██████████| 161.263/161.263 [00:08<00:00, 18.18s/s]           


time_taken 9.296969890594482


# tests

In [1]:
from pytube import YouTube
yt = YouTube("https://www.youtube.com/watch?v=MP7tdlNM3pw")
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=output_path, filename=filename)

NameError: name 'output_path' is not defined

In [19]:
yt.streams.filter(file_extension='mp4').order_by('resolution').desc().first().download()

'/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/Traversée de l’Atlantique en ballon stratosphérique XXL  Actu de science.mp4'